In [218]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from requests import get
from bs4 import BeautifulSoup
import json

In [99]:
%%time
chrome_prefs = {'profile.default_content_settings': {'images': 2},
                'profile.managed_default_content_settings': {'images': 2}
               }
option = webdriver.ChromeOptions()
option.experimental_options["prefs"] = chrome_prefs
browser = webdriver.Chrome(executable_path = 'C:\\Users\\nahue\\Desktop\\chromedriver.exe',
                          chrome_options = option
                          )
browser.get('https://www.monsieur-cuisine.com/nc/en/search/')
time.sleep(2)
cookie = browser.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[1]/div[2]/button[2]')
cookie.click()
time.sleep(2)
elem = browser.find_element_by_tag_name('body')
for i in range(5):
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
post_elems = browser.find_elements_by_class_name('recipe-card-link')
links = []
for post in post_elems:
    links = links + [post.get_attribute("href")]

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


Wall time: 27.4 s


In [100]:
len(links)

39

In [223]:
%%time
names = []
ingreds = []
instrs = []
for i in range(len(links)):
    i = links[i]
    response = get(i)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    name = html_soup.find('h1', class_ = 'red').text
    names = names + [name]
    ingredients = html_soup.find('div',
                                 class_ = 'recipe--ingredients-html-item col-md-8'
                                ).find_all('li')
    ingred = list(range(len(ingredients)))
    for i in range(len(ingredients)):
        ingred[i] = ingredients[i].text[2:]
    ingreds = ingreds + [ingred]
    instructions = ingredients = html_soup.find('div',
                                 id = 'recipeInstruction'
                                ).ol.find_all('li')
    
    instr = list(range(len(instructions)))
    for i in range(len(instructions)):
        instr[i] = instructions[i].text
    instrs = instrs + [instr]
recipe = {}
for i in range(len(names)):
    recipe[names[i]] = {}
    recipe[names[i]]['Ingredients'] = ingreds[i]
    recipe[names[i]]['Instructions'] = instrs[i]

{'Sausage goulash': {'Ingredients': ['2 shallots (60 g)', '1 garlic clove', '2 tbsp olive oil', '1 tbsp honey', '1 tbsp tomato purée', '30 ml apple cider vinegar', '3 tbsp soy sauce', '500 g sieved tomatoes (passata)', '2 pinches of salt', '3 pinches of pepper', '1 tbsp curry', '40 ml pineapple juice', '400 g bockwurst sausages'], 'Instructions': ['Peel and halve the shallots. Peel the garlic cloves and put in the blender jug with the shallots. With the measuring beaker in place, chop for 5 seconds/speed setting 6. Use the spatula to scrape the pieces down the inside of the blender jug. Then add the olive oil. The measuring beaker not inserted, stew for 2 minutes/100 °C/speed setting 1.', 'Add the honey, tomato purée, apple cider vinegar, soy sauce, passata, 2 pinches of salt, 3 pinches of pepper, curry and pineapple juice, and, with the measuring beaker in place, cook for 15 minutes/100 °C/speed setting 2.', 'Cut the sausages into 1.5-cm slices and add to the sauce. With the measuring

In [229]:
f = open('C:\\Users\\nahue\\Desktop\\Projects\\Recipes.txt', 'w')
f.write(json.dumps(recipe,
                   sort_keys = True,
                   indent = 4
                  )
       )

60877